<a href="https://colab.research.google.com/github/archit436/Birds_Classifier/blob/main/DP_Stage2_XC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

0. Setup


In [22]:
# Import Relevant Libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import requests
import os
import librosa
import soundfile as sf
from pydub import AudioSegment

1. Data Acquisiton


In [23]:
# Mount the google drive
from google.colab import drive
drive.mount('/content/drive')
csv_path = "/content/drive/MyDrive/APS360_Team_15/Data_Processing/chosen_classes_data_stats.csv"

# We start by importing the information about birds outputted by the stage 1 in
# the form of a csv file.
birds_df = pd.read_csv(csv_path)
birds_df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Class ID,Images Count,XC Recordings Count,Species Name,XC Species Name
0,315,116,167,Gadwall (Breeding male),Gadwall
1,317,120,240,Mallard (Breeding male),Mallard
2,333,105,114,Common Goldeneye (Breeding male),Common Goldeneye
3,338,116,89,California Quail (Male),California Quail
4,352,120,285,Black-crowned Night-Heron (Adult),Black-crowned Night Heron


In [ ]:
# We will now query into Xeno-Canto API to download the audio files for the birds.
# We create a directory called Xeno Canto to store all the audio files.
xeno_canto_dir = "/content/drive/MyDrive/APS360_Team_15/Data/Xeno_Canto"
os.makedirs(xeno_canto_dir, exist_ok=True)

# Iterate through all the birds manually, due to timeout issues with Xeno Canto
it_bird = 106

# Extract basic data about the bird from the dataframe.
bird_id = birds_df["Class ID"][it_bird]
bird_xc_name = birds_df["XC Species Name"][it_bird]
bird_xc_recordings_count = birds_df["XC Recordings Count"][it_bird]

# Make a subdirectory for the current bird using its Class ID.
bird_dir = os.path.join(xeno_canto_dir, str(bird_id))
os.makedirs(bird_dir, exist_ok=True)

# Get the bird name and query the Xeno-Canto API.
# As before, we only query for high quality recordings
search_query = "+".join(bird_xc_name.split())
encoded_search_query = f"{search_query}+q:A"
url = f"https://www.xeno-canto.org/api/2/recordings?query={encoded_search_query}"
search_response = requests.get(url)
search_data = search_response.json()

# Use the search query response to download all the audio files.
# We limit the recordings to 500 per species, if they have more available.
for it_rec in range(min(500,bird_xc_recordings_count)):
    audio_url = search_data["recordings"][it_rec]["file"]
    audio_response = requests.get(audio_url, stream=True)
    with open(f"{bird_dir}/{it_rec}.mp3", "wb") as f:
        f.write(audio_response.content)
print(f"Downloaded {min(500,bird_xc_recordings_count)} recordings for {bird_xc_name}.")

Downloaded 337 recordings for Red-winged Blackbird.


In [25]:
# Code to reacquire any faulty recordings.
bird_class_id = 382
file_id = 19

# Find the name corresponding this bird.
bird_name = birds_df[birds_df["Class ID"] == bird_class_id]["XC Species Name"].values[0]

# Make the query and get the search response.
search_query = "+".join(bird_name.split())
encoded_search_query = f"{search_query}+q:A"
url = f"https://www.xeno-canto.org/api/2/recordings?query={encoded_search_query}"
search_response = requests.get(url)
search_data = search_response.json()

# Retrive the audio file and write to a file.
audio_url = search_data["recordings"][file_id]["file"]
audio_response = requests.get(audio_url, stream=True)

# Define the directory to write to and then download the audio file.
xeno_canto_dir = "/content/drive/MyDrive/APS360_Team_15/Data/Xeno_Canto"
bird_dir = os.path.join(xeno_canto_dir, str(bird_class_id))
with open(f"{bird_dir}/{file_id}.mp3", "wb") as f:
    f.write(audio_response.content)
print(f"Downloaded {file_id}.mp3 for {bird_name}.")
print(search_data["recordings"][19])

Downloaded 19.mp3 for Northern Flicker.
{'id': '713588', 'gen': 'Colaptes', 'sp': 'auratus', 'ssp': 'Colaptes auratus auratus/luteus', 'group': 'birds', 'en': 'Northern Flicker', 'rec': 'David A. Brinkman', 'cnt': 'United States', 'loc': 'Miami Township (near  North Bend), Hamilton County, Ohio', 'lat': '39.1317', 'lng': '-84.7973', 'alt': '160', 'type': 'song', 'sex': 'male', 'stage': 'adult', 'method': 'field recording', 'url': '//xeno-canto.org/713588', 'file': 'https://xeno-canto.org/713588/download', 'file-name': 'XC713588-NOFL SL Boat Ramp Mar 23 2022.mp3', 'sono': {'small': '//xeno-canto.org/sounds/uploaded/IUWJMMZJNL/ffts/XC713588-small.png', 'med': '//xeno-canto.org/sounds/uploaded/IUWJMMZJNL/ffts/XC713588-med.png', 'large': '//xeno-canto.org/sounds/uploaded/IUWJMMZJNL/ffts/XC713588-large.png', 'full': '//xeno-canto.org/sounds/uploaded/IUWJMMZJNL/ffts/XC713588-full.png'}, 'osci': {'small': '//xeno-canto.org/sounds/uploaded/IUWJMMZJNL/wave/XC713588-small.png', 'med': '//xeno-ca